In [1]:
import numpy as np
import pandas as pd

In [37]:
import os
import tqdm

In [7]:
DATAPATH = "../data/fantasy-data-nba-regular"
os.listdir(DATAPATH)

['fantasy-data-nba-regular-2015.csv',
 'fantasy-data-nba-regular-2014.csv',
 'fantasy-data-nba-regular-2016.csv',
 'fantasy-data-nba-regular-2017.csv',
 'fantasy-data-nba-regular-2013.csv',
 'fantasy-data-nba-regular-2012.csv',
 'fantasy-data-nba-regular-2010.csv',
 'fantasy-data-nba-regular-2011.csv',
 'fantasy-data-nba-regular-2020.csv',
 'fantasy-data-nba-regular-2021.csv',
 'fantasy-data-nba-regular-2009.csv',
 'fantasy-data-nba-regular-2022.csv',
 'fantasy-data-nba-regular-2019.csv',
 'nba-fantasy-regular-columns.csv',
 'fantasy-data-nba-regular-2018.csv']

In [27]:
df2009 = pd.read_csv(DATAPATH+"/fantasy-data-nba-regular-2009.csv")
df2009.drop(inplace=True,columns=['RK','NAME','TEAM','POS','FPTS'])

df2009.head()

,GMS,PTS,REB,AST,BLK,STL,FGP,FTP,3PP,FTM,2PM,3PM,TO,MIN,DD2,TD3
0,81,2304,613,587,93,137,48.9,78.0,34.4,594,657,132,241,3054,29,7
1,79,2386,398,589,106,173,49.1,76.5,31.7,590,766,88,272,3048,20,0
2,78,1781,432,861,10,216,50.3,86.8,36.4,455,567,64,231,3002,50,6
3,79,1624,1093,112,231,77,57.2,59.4,0.0,504,560,0,240,2821,63,1
4,82,2201,429,399,37,120,46.7,85.6,35.1,483,682,118,210,2960,8,2


# feature normalization steps
* discretize name, team, pos (one-hot encoding)
* split data into train/test
* normalize train data and test data based on train data stats
* save the dataset

In [ ]:
df2009.NAME.unique().shape

(437,)

In [ ]:
df2009.TEAM.unique().shape

(30,)

TODO
- [ ] train basic ranking model with decision trees
- [ ] run a linear regression model with perceptron
- [ ]

In [ ]:
df2009.shape

(438, 21)

In [ ]:
438*437

191406

In [ ]:
rows, cols = df2009.shape
data = []
for rowi in range(rows):
    row1 = df2009.iloc[rowi,:].to_numpy()
    for rowj in range(rows):
        if rowi != rowj:
            row2 = df2009.iloc[rowj,:].to_numpy()
            transformed_row = np.concatenate([row1, row2, np.array([1]) if rowi < rowj else np.array([-1])])
            data.append(transformed_row)
data = np.array(data)
print(data.shape)

(191406, 33)


In [ ]:
print(data)

[[ 8.100e+01  2.304e+03  6.130e+02 ...  2.000e+01  0.000e+00  1.000e+00]
 [ 8.100e+01  2.304e+03  6.130e+02 ...  5.000e+01  6.000e+00  1.000e+00]
 [ 8.100e+01  2.304e+03  6.130e+02 ...  6.300e+01  1.000e+00  1.000e+00]
 ...
 [ 1.000e+00  0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00 -1.000e+00]
 [ 1.000e+00  0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00 -1.000e+00]
 [ 1.000e+00  0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00 -1.000e+00]]


In [77]:
np.random.shuffle(data)

In [78]:
num_samples = data.shape[0]
num_train = int(num_samples*0.5)
print(num_train)

95703


In [79]:
X_train = data[:num_train,:-1]
y_train = data[:num_train,-1]
X_test = data[num_train:,:-1]
y_test = data[num_train:,-1]

In [80]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((95703, 32), (95703,), (95703, 32), (95703,))

In [81]:
from sklearn import tree, linear_model

In [82]:
model = linear_model.Perceptron()

In [83]:
model = model.fit(X_train, y_train)

In [84]:
model.classes_

array([-1.,  1.])

In [85]:
model.score(X_test, y_test)

0.9975444865887172

# big loop

## data preparation

In [86]:
data = []
for year in tqdm.tqdm(range(2009, 2022)):
    df = pd.read_csv(DATAPATH+"/fantasy-data-nba-regular-{}.csv".format(year))
    df.drop(inplace=True,columns=['RK','NAME','TEAM','POS','FPTS'])
    rows, cols = df.shape
    for rowi in range(rows):
        row1 = df.iloc[rowi,:].to_numpy()
        for rowj in range(rows):
            if rowi != rowj:
                row2 = df.iloc[rowj,:].to_numpy()
                transformed_row = np.concatenate([row1, row2, np.array([1]) if rowi < rowj else np.array([-1])])
                data.append(transformed_row)
data = np.array(data)

KeyboardInterrupt: 